In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('talk', font_scale=1.2, rc={'lines.linewidth': 3})
sns.set_style('whitegrid',
              {'grid.linestyle': ':', 'grid.color': 'red', 'axes.edgecolor': '0.5',
               'axes.linewidth': 1.2, 'legend.frameon': True})

In [2]:
from scipy.constants import m_p, c, e

In [3]:
from cpymad.madx import Madx

import sixtracklib as pyst
import pysixtrack

# I. MAD-X part

In [4]:
madx = Madx()
madx.options.echo = False
madx.options.warn = False
madx.options.info = False


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2019.10.09 18:22:30      +
  ++++++++++++++++++++++++++++++++++++++++++++


In [5]:
p0c = 1e9 # in eV
Etot = np.sqrt(p0c**2 + (m_p/e)**2 * c**4) * 1e-9 # in GeV

Define a simple sequence in MAD-X:

In [6]:
madx.input('''
qd: multipole, knl := {0, 0.1};

line: sequence, l = 1;
qd, at = 0.5;
endsequence;
''')

madx.command.beam(particle='proton', energy=Etot) # energy in GeV

madx.use(sequence='line')

## I.1. track through `line` without tilt error:

tracking:

In [7]:
madx.command.track(onetable=True)
madx.input('NPART=1;')
madx.command.start(x=0.01, y=0.005)
madx.command.run()
madx.command.endtrack()

enter TRACK module
exit TRACK module



True

relative change of $x$ coordinate:

In [8]:
x_noerr = madx.table.trackone[1]['x'] / madx.table.trackone[0]['x']
x_noerr

0.9499999687499706

## I.2. now track with tilt error:

In [9]:
dpsi = 1.0

Apply this `dpsi` tilt error to the quadrupole:

In [10]:
madx.select(flag='error', pattern='qd')
madx.command.ealign(dpsi=dpsi)

qd = madx.sequence.line.expanded_elements[2]

qd.align_errors.dpsi

1.0

tracking:

In [11]:
madx.command.track(onetable=True)
madx.input('NPART=1;')
madx.command.start(x=0.01, y=0.005)
madx.command.run()
madx.command.endtrack()

enter TRACK module
exit TRACK module



True

relative change of $x$ coordinate:

In [12]:
x_err = madx.table.trackone[1]['x'] / madx.table.trackone[0]['x']
x_err

0.9980749049535305

In comparison to no error:

In [13]:
x_err / x_noerr

1.0506051976683934

$\implies$ significant effect of `dpsi` tilt error on x coordinate

# II. SixTrackLib part

## II.1. without tilt error:

In [14]:
elements = pyst.Elements.from_mad(madx.sequence.line, exact_drift=True)
elements.BeamMonitor(num_stores=1);

In [15]:
particles = pyst.Particles.from_ref(1, p0c=p0c)

In [16]:
particles.x = 0.01
particles.y = 0.005

In [17]:
job = pyst.TrackJob(elements, particles)

In [18]:
job.track(1)

job.collect()

job.output.particles[0]

<Particles at 128
  num_particles:1
  q0:[1.]
  mass0:[9.38272081e+08]
  beta0:[0.72925621]
  gamma0:[1.46147393]
  p0c:[1.e+09]
  s:[1.]
  x:[0.0095]
  y:[0.00525]
  px:[-0.001]
  py:[0.0005]
  zeta:[-3.12500293e-07]
  psigma:[0.]
  delta:[0.]
  rpp:[1.]
  rvv:[1.]
  chi:[1.]
  charge_ratio:[1.]
  particle_id:[0]
  at_element:[5]
  at_turn:[0]
  state:[1]
>

In [19]:
particles.x[0] / 0.01

0.9499999687499706

Comparing to `x_noerr` from the MAD-X tracking above:

In [20]:
x_noerr == particles.x[0] / 0.01

True

$\implies$ `SixTrackLib` and `MAD-X` agree exactly on the tracking for the single particle!

## II.2. with tilt error:

Create Line in `PySixTrack` in order to manipulate it and add the `SRotation` element for the tilt:

In [21]:
pysixtrack_elements, _ = pysixtrack.line.Line.from_madx_sequence(madx.sequence.line)

In [22]:
pysixtrack_elements.elements

[Drift(length=0.0),
 Drift(length=0.5),
 Multipole(knl=[0.0, 0.1], ksl=[0.0], hxl=0.0, hyl=0, length=0.0),
 Drift(length=0.5),
 Drift(length=0.0)]

Define the tilt in `PySixTrack`:

In [23]:
angle = dpsi * 180 / np.pi

srot = pysixtrack.elements.SRotation(angle=angle)
inv_srot = pysixtrack.elements.SRotation(angle=-angle)

In [24]:
pysixtrack_elements.elements.insert(2, srot)
pysixtrack_elements.elements.insert(4, inv_srot)
pysixtrack_elements.elements

[Drift(length=0.0),
 Drift(length=0.5),
 SRotation(angle=57.29577951308232),
 Multipole(knl=[0.0, 0.1], ksl=[0.0], hxl=0.0, hyl=0, length=0.0),
 SRotation(angle=-57.29577951308232),
 Drift(length=0.5),
 Drift(length=0.0)]

Pass over to `SixTrackLib`:

In [25]:
elements = pyst.Elements.from_line(pysixtrack_elements)
elements.BeamMonitor(num_stores=1);

In [26]:
particles = pyst.Particles.from_ref(1, p0c=p0c)

In [27]:
particles.x = 0.01
particles.y = 0.005

In [28]:
job = pyst.TrackJob(elements, particles)

In [29]:
job.track(1)

job.collect()

job.output.particles[0]

<Particles at 128
  num_particles:1
  q0:[1.]
  mass0:[9.38272081e+08]
  beta0:[0.72925621]
  gamma0:[1.46147393]
  p0c:[1.e+09]
  s:[1.]
  x:[0.00998075]
  y:[0.00444131]
  px:[-3.85018769e-05]
  py:[-0.00111737]
  zeta:[-3.125e-07]
  psigma:[0.]
  delta:[0.]
  rpp:[1.]
  rvv:[1.]
  chi:[1.]
  charge_ratio:[1.]
  particle_id:[0]
  at_element:[7]
  at_turn:[0]
  state:[1]
>

In [30]:
particles.x[0] / 0.01

0.9980749061567152

Comparing to `x_err` from the MAD-X tracking above:

In [31]:
x_err

0.9980749049535305

Almost identical...